(chap:csc-data)=
# Retrieving X-ray Data from the `CSC`

For my work, the most important information that I need from *CXO* is the coordinates of the X-ray sources (used to locate their optical counterparts on an *HST* image), their X-ray luminosities (calculated from the `CSC`-provided flux), their hardness ratios (used to determine whether or not they may be a supernova remnant in a later section), the off-axis angle, and the total X-ray counts (the last two of which are used to calculate the uncertainty radii in {ref}`chap:astrometry`). This information can be pulled directly from `CSCView.jar`.

Open the program by clicking the icon or typing into the command line (where it is saved): 
```
java -jar cscview.jar &
```
(The optional `&` allows you to continue using the terminal for other processes while `cscview.jar` is running)

This will open the program GUI, which will look something like: 

```{image} ../images/cscview.png
:name: fig-cscview

Example search using the CSCView desktop app.
```

Select the most recent data release. Under `Source Properties`, search for hardness ratio or `hard_hs` and add it to the Result Set with the `+` button. Do the same for off-axis angle (`theta` or `theta_mean`) and total X-ray counts ({cite:p}`lehmer19` appears to use `src_cnts_aper_b`, `src_cnts_aper_lolim_b`, and `src_cnts_aper_hilim_b`).
All other results can be left as is, or you may remove the properties that you won't need to save memory. You can save the query for future use by clicking the `Save` button at the top of the window. By default, it is saved with the suffix `.prop`.

```{note}
There may be additional data needed in order to clean up the X-ray source catalog such that only good data are included in your analysis. Since I focused mainly on X-ray sources identified in {cite:p}`lehmer19`, which did this step for me, I never had to deal with this. However, it's something that will need to be kept in mind for future products. See {cite:p}`lehmer19` for more information on data selection from `CSC`.
```

To search for the *CXO*  X-ray sources associated with your galaxy of interest, select `Cone` under `Position Search` at the bottom of the window and search for the galaxy of interest by name or coordinates. In the example above, I am looking for X-ray sources in M101, which has a linear diameter of roughly 30 arcminutes. I set the search parameters accordingly and hit the `Search` button at the top left of the window. This automatically brings you from the `Query` tab to the `Results` tab, from which you can select all the X-ray sources and save them to a `.vot` file. 

```{note}
You will need to press the `Search` button every time you make a change in the `Query` tab. Otherwise, your search results may not be updated!
```

To view the `.vot` file, open `TopCat`: 
```
java -jar topcat-full.jar
```
(or whatever equivalent you saved it under). This will open a window that resembles the following: 



```{figure} ../images/topcat.png
:name: fig-topcat

Example of the TOPCAT user interface.
```


In the GUI above, navigate to the `.vot` file via the `Filestore Browser` or `System Browser` interface. This will pull up the file information under `Current Table Properties`. The 4th icon at the top of the window (looks like a spreadsheet) will open the `Table Browser`, which allows you to view the table as a spreadsheet. 

(sec:csc-dataframe)=
## Converting from `VOTable` to Pandas `DataFrame`

In the repository for this guide, I've included custom code that I use to convert `.vot` files into Pandas `DataFrames` in `python`, a function called `NewSources()` from the `Sources.py` script. To use this function, you simply need to open run something like this in an iPython notebook:


In [2]:
# When running python from a notebook, it's good to import a function
# that will allow you to navigate your directories
import os
cd = os.chdir
pwd = os.getcwd

from XRBID.Sources import NewSources

NewSources("testdata/cscresults_M101.vot") # I saved my .vot to a directory called 'testdata'


ModuleNotFoundError: No module named 'XRBID'

This will read in the table exactly as written. Keep in mind, the code will need to point towards where `Sources.py` and your `.vot` file(s) are saved on your computer! If you want to rename the headers or save the `Dataframe` to a file, you can do so using: 

```
NewSources("testdata/cscresults_M101.vot", rename=True, \
           outfile="testdata/cscresults_M101.frame")
```

```{figure} ../images/newsources_output.png
:width: 500px
:align: center
```

The code above will prompt you to rename each of the headers within the `.vot` Table, then save the `DataFrame` as a comma-separated value that is easily read in by `pd.read_csv()` or, better yet, my custom function `Sources.LoadSources()`. The latter is a little slower, but it will automatically remove the index column of the `CSV` file and convert applicable values from strings into actual numbers as the `DataFrame` is being built. This is usually very helpful during the analysis stage, so I recommend using `LoadSources()` as long as it's working properly. 

You should also save these sources as a region file that can be loaded into `DS9` or `CARTA` to overlay onto an *HST* `FITS` file. This can be done fairly simply with `WriteScript.WriteReg()`. 